<a href="https://colab.research.google.com/github/vishnuvsgthn/Classification/blob/main/Traning__data_aug_Data_Analysis_Vgg16_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd drive/MyDrive/Crayon

/content/drive/MyDrive/Crayon


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
#Finding the number of images for train test and validation
print("Number of images in traning covid p folder")
!ls ./temp/Train_Augment/Augment/covid_p/ | wc -l
print("Number of images in traning covid_n folder")
!ls ./temp/Train_Augment/Augment/covid_n/ | wc -l

print("Number of images in validation covid p folder")
!ls ./temp/Validation/covid_p/ | wc -l
print("Number of images in validation covid_n folder")
!ls ./temp/Validation/covid_n/ | wc -l

print("Number of images in test covid p folder")
!ls ./temp/Test/covid_p/ | wc -l
print("Number of images in test covid_n folder")
!ls ./temp/Test/covid_n/ | wc -l



Number of images in traning covid p folder
735
Number of images in traning covid_n folder
1540
Number of images in validation covid p folder
15
Number of images in validation covid_n folder
30
Number of images in test covid p folder
30
Number of images in test covid_n folder
60


In [28]:
import sklearn
import os 
import zipfile 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

In [ ]:
# local_zip = './Dataset/dataset.zip'
# zip_ref = zipfile.ZipFile(local_zip, 'r')
# zip_ref.extractall('./temp')
# zip_ref.close()

In [30]:
base_dir = './temp'
base_dir_train = './temp/Train_Augment'
train_dir = os.path.join(base_dir_train, 'Augment')
validation_dir = os.path.join(base_dir, 'Validation')

# Directory with our training cat pictures
train_cneg_dir = os.path.join(train_dir, 'covid_n')

# Directory with our training dog pictures
train_cpos_dir = os.path.join(train_dir, 'covid_p')

# Directory with our validation cat pictures
validation_cneg_dir = os.path.join(validation_dir, 'covid_n')

# Directory with our validation dog pictures
validation_pos_dir = os.path.join(validation_dir, 'covid_p')


In [31]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 7, width_shift_range = 0.2, height_shift_range = 0.2,shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True,brightness_range=[0.4,1.5])

test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [32]:
train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 16, class_mode = 'binary', target_size = (224, 224))
validation_generator = test_datagen.flow_from_directory(validation_dir, batch_size = 8, class_mode = 'binary', target_size = (224, 224))

Found 2275 images belonging to 2 classes.
Found 45 images belonging to 2 classes.


In [33]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(class_weight = "balanced", classes= np.unique(train_generator.classes), y= train_generator.classes)
#class_weights = dict((np.unique(train_generator.classes), class_weights)),
#class_weights =  class_weights[0]
class_weights = {0: class_weights[0], 1:class_weights[1]}
#class_ids = list(sorted(class_weights.keys()))
#class_ids


In [34]:
#addign vgg16
from tensorflow.keras.applications import VGG16

base_model = VGG16(input_shape=(224, 224,3), include_top=False, weights="imagenet")

In [35]:
for layer in base_model.layers:
    layer.trainable = True

In [36]:
from tensorflow.keras.applications import VGG16
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

base_model = Sequential()
base_model.add(VGG16(include_top=False, weights='imagenet', pooling='max'))
base_model.add(Dense(1, activation='sigmoid'))

In [37]:
save_path ='./content/drive/MyDrive/Crayon/Models/VGG16'
checkpoint = ModelCheckpoint(save_path, verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  
optimizer = Adam(learning_rate=0.0001)
base_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])

In [38]:
train_batch_size = 16
val_batch_size = 8
steps_per_epoch = train_generator.samples//train_generator.batch_size
val_steps = validation_generator.samples//validation_generator.batch_size
print(val_steps)


resnet_history = base_model.fit(train_generator, validation_data = validation_generator,steps_per_epoch=steps_per_epoch,validation_steps=val_steps ,callbacks=[checkpoint], epochs = 25,class_weight = class_weights,verbose=1)

5
Epoch 1/25
142/142 [==============================] - 563s 4s/step - loss: 0.7383 - acc: 0.5229 - val_loss: 0.5220 - val_acc: 0.6750

Epoch 1: val_loss improved from inf to 0.52203, saving model to ./content/drive/MyDrive/Crayon/Models/VGG16
INFO:tensorflow:Assets written to: ./content/drive/MyDrive/Crayon/Models/VGG16/assets
Epoch 2/25
142/142 [==============================] - 63s 435ms/step - loss: 0.5574 - acc: 0.7353 - val_loss: 0.4810 - val_acc: 0.8000

Epoch 2: val_loss improved from 0.52203 to 0.48101, saving model to ./content/drive/MyDrive/Crayon/Models/VGG16
INFO:tensorflow:Assets written to: ./content/drive/MyDrive/Crayon/Models/VGG16/assets
Epoch 3/25
142/142 [==============================] - 62s 436ms/step - loss: 0.5241 - acc: 0.7627 - val_loss: 0.4478 - val_acc: 0.8250

Epoch 3: val_loss improved from 0.48101 to 0.44784, saving model to ./content/drive/MyDrive/Crayon/Models/VGG16
INFO:tensorflow:Assets written to: ./content/drive/MyDrive/Crayon/Models/VGG16/assets
Ep

In [40]:
import pandas as pd
hist_df = pd.DataFrame(resnet_history.history) 
hist_csv_file = 'vgg_16_train_history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)


In [ ]:
#apply the augmentation
!pip install clodsa


     |████████████████████████████████| 5.7 MB 4.8 MB/s 
     |████████████████████████████████| 276 kB 59.6 MB/s 
  Created wheel for clodsa: filename=clodsa-1.2.47-py2.py3-none-any.whl size=74310 sha256=9f622a1d7bc72edd6ac47acd3184195f5529ce167af553a3fc1e8144d0d97a02
  Stored in directory: /root/.cache/pip/wheels/05/ff/0a/0e6e14c2a68d6869a010e979b8fd9d669aaeaa2d8b29de394f
  Created wheel for commentjson: filename=commentjson-0.9.0-py3-none-any.whl size=12092 sha256=3b9a14ff644a49453d86ffbdb7b3e12bfc0d2440b8aa09853e047a837f79e0a0
  Stored in directory: /root/.cache/pip/wheels/eb/bb/07/25a7f0718ee3fe137384011b8e56070f91cf950ee6047c287f
  Created wheel for lark-parser: filename=lark_parser-0.7.8-py2.py3-none-any.whl size=62527 sha256=297e4cef6eb114dea19f164a50ea20f5eb27343b4334ca87eb691e4ed2e7bb5e
  Stored in directory: /root/.cache/pip/wheels/92/e3/af/1dc0fdca93232d700ac176af6554cf22b85f3d7e8aeee5ac08
Successfully built clodsa commentjson lark-parser


In [ ]:
from matplotlib import pyplot as plt
from clodsa.augmentors.augmentorFactory import createAugmentor
from clodsa.transformers.transformerFactory import transformerGenerator
from clodsa.techniques.techniqueFactory import createTechnique
import cv2
%matplotlib inline

In [ ]:
PROBLEM = "classification"
ANNOTATION_MODE = "folders"
INPUT_PATH = "./temp/Train/"
GENERATION_MODE = "linear"
OUTPUT_MODE = "folders"
OUTPUT_PATH= "./temp/Train/Augment/"
augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH})
transformer = transformerGenerator(PROBLEM)
hFlip = createTechnique("flip",{"flip":1})
augmentor.addTransformer(transformer(hFlip))
crop = createTechnique("crop",{"percentage":0.9,"startFrom": "TOPLEFT"})
augmentor.addTransformer(transformer(crop))
translation_pos = createTechnique("translation", {"x":10,"y":10})
augmentor.addTransformer(transformer(translation_pos))
avgBlur =  createTechnique("average_blurring", {"kernel" : 5})
augmentor.addTransformer(transformer(avgBlur))
equalize = createTechnique("equalize_histogram",{})
augmentor.addTransformer(transformer(equalize))
salt_and_pepper = createTechnique("salt_and_pepper",{"low":0,"high":255})
augmentor.addTransformer(transformer(salt_and_pepper))
none = createTechnique("none",{})
augmentor.addTransformer(transformer(none))
augmentor.applyAugmentation()



In [ ]:
# !rm -r ./temp/Train/Augment

In [ ]:
%cd temp/Train/Augment

/content/drive/My Drive/Crayon/temp/Train/Augment


In [ ]:
!ls

covid_n  covid_p


In [ ]:
#Finding the number of images for train test and validation
print("Number of images in traning covid p folder")
!ls ./covid_p/ | wc -l
print("Number of images in traning covid_n folder")
!ls ./ | wc -l





Number of images in traning covid p folder
1785
Number of images in traning covid_n folder
2
